In [1]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers
!pip install -q -U "huggingface_hub[cli]"

In [44]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName="sagemaker_execution_role")['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::395271362395:role/SagemakerStudioDemoSagema-SageMakerExecutionRole78-5I33I083KE6P
sagemaker bucket: sagemaker-us-east-1-395271362395
sagemaker session region: us-east-1


In [46]:
from sagemaker.s3 import S3Downloader

downloader = S3Downloader()

In [47]:
model_output = 's3://sagemaker-us-east-1-395271362395/gemma-9b-text-to-sql-2024-07-18-08-47-19-847/output/model'
downloader.download(model_output, './model', sagemaker_session=sess)

['./model/config.json',
 './model/generation_config.json',
 './model/model-00001-of-00010.safetensors',
 './model/model-00002-of-00010.safetensors',
 './model/model-00003-of-00010.safetensors',
 './model/model-00004-of-00010.safetensors',
 './model/model-00005-of-00010.safetensors',
 './model/model-00006-of-00010.safetensors',
 './model/model-00007-of-00010.safetensors',
 './model/model-00008-of-00010.safetensors',
 './model/model-00009-of-00010.safetensors',
 './model/model-00010-of-00010.safetensors',
 './model/model.safetensors.index.json',
 './model/special_tokens_map.json',
 './model/tokenizer.json',
 './model/tokenizer.model',
 './model/tokenizer_config.json']

In [12]:
import torch

try:
    del model
    del tokenizer
except:
    pass

torch.cuda.empty_cache()

In [13]:
!nvidia-smi

Fri Jul 19 01:22:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   32C    P0              65W / 300W |    334MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

#model_id = 'google/gemma-2-9b'
model_id = './model'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    add_eos_token=True,
)

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [16]:
def generate_prompt(schema: str, query: str):
    prompt = f"""
<start_of_turn>user
You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA: {schema}

{query}<end_of_turn>
<start_of_turn>model
""".lstrip()
    return prompt

In [17]:
def get_completion(schema, query, model, tokenizer) -> str:
    prompt = generate_prompt(schema, query)
    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return (decoded)

In [18]:
# SELECT name, born_state, age FROM head ORDER BY age

result = get_completion(
    schema="CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR);",
    query="List the name, born state and age of the heads of departments ordered by age.",
    model=model,
    tokenizer=tokenizer,
)
print(result)

user
You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA: CREATE TABLE head (name VARCHAR, born_state VARCHAR, age VARCHAR);

List the name, born state and age of the heads of departments ordered by age.
model
 head FROM head JOIN ( SELECT head FROM head WHERE department LIKE '%head%') AS heads_of_departments ON heads_of_departments.born_state IS NOT NULL;
model
SELECT name, born_state, age FROM heads_of_departments;
model
SELECT name, born_state, age FROM ( SELECT data.name FROM data JOIN (SELECT * FROM data WHERE department LIKE '%head%') AS head FROM data WHERE head EXISTS ) AS heads_of_departments;
model
SELECT name, born_state, age FROM heads_of_departments;
model
SELECT name, born_state, age FROM (SELECT data.name FROM data JOIN (SELECT * FROM data WHERE department LIKE '%head%') AS head FROM data WHERE head EXISTS) AS heads_of_departments;
model
SELECT name, born_state, age FROM h

In [19]:
# SELECT name FROM head WHERE born_state <> 'California'

result = get_completion(
    schema="CREATE TABLE head (name VARCHAR, born_state VARCHAR);",
    query="What are the names of the heads who are born outside the California state?",
    model=model,
    tokenizer=tokenizer,
)
print(result)

user
You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA: CREATE TABLE head (name VARCHAR, born_state VARCHAR);

What are the names of the heads who are born outside the California state?
model
SELECT name FROM head WHERE born_state!= 'California';
model
SELECT name FROM head WHERE birth_location != 'California';
model
SELECT name FROM head WHERE state != 'California';
model
SELECT name FROM head WHERE state != 'California';
model
SELECT name FROM head WHERE birth_date >= CURRENT_DATE - INTERVAL (SELECT EXTRACT(CURRENT_DATE FROM head) - birth_date) * 10);
model
SELECT name FROM head WHERE birth_date <= CURRENT_DATE + INTERVAL '10 years';
model
SELECT name FROM head WHERE birth_date >= CURRENT_DATE - INTERVAL '20 years';
model
SELECT name FROM head WHERE birth_date >= CURRENT_DATE - INTERVAL '30 years';
model
SELECT name FROM head WHERE birth_date >= CURRENT_DATE - INTERVAL '50 years';
mo